In [13]:
import netket as nk

In [14]:
hilb = nk.hilbert.SpinOrbitalFermions(
    n_orbitals=27, s=0, n_fermions=18
)

In [ ]:
hilb.states

[0, 1]

In [5]:
hilb.n_states

4686825

In [ ]:
for state in hilb.all_states():
    print(state)
    break

KeyboardInterrupt: 